In [1]:
import tensorflow as tf
import tflearn
import numpy as np
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from random import shuffle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# Defining constants
#ver 1 = 10 epochs
IMG_SIZE = 54 # 50 x 50 image, for example
TIME_LEN = 10 # 5 time points, for example
LR = 1e-3 # learning rate, for example
CONV_LAYERS = 1
POOL_LAYERS = 1
NO_CHANNELS = 1 # how many input streams i.e. a network with 3 channels could have RGB, depth, and body skeleton streams
VERSION = 2 # to keep track of changes in a particular model
NO_SIGNS = 3 # number of signs to learn - i.e. the alphabet
N_EPOCH = 5 # number of epochs
MODEL_NAME = '3dconv-{}--{}--{}--{}--{}-.model'.format(LR, CONV_LAYERS, POOL_LAYERS, NO_CHANNELS, VERSION)

In [4]:
train_data = np.load("trainingData.npy")
shuffle(train_data)

X = np.array([i[0] for i in train_data]).reshape([-1, 10, 54, 54, 1])
Y = [i[1] for i in train_data]

In [5]:
conv_net = input_data(shape = [None, TIME_LEN, IMG_SIZE, IMG_SIZE, NO_CHANNELS])

conv_net = conv_3d(conv_net, 32, 3, strides = 1, padding = 'same', activation = 'relu')

conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = fully_connected(conv_net, 256, activation = 'relu')

conv_net = dropout(conv_net, 0.8)

conv_net = fully_connected(conv_net, NO_SIGNS, activation = 'softmax')

conv_net = regression(conv_net, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model = tflearn.DNN(conv_net, tensorboard_dir = 'log', tensorboard_verbose=3)

In [7]:
model.fit(X, Y, n_epoch = N_EPOCH, validation_set = 0.1, show_metric = True, run_id = MODEL_NAME) 


Training Step: 129  | total loss: 4.29591 | time: 283.186s
| Adam | epoch: 010 | loss: 4.29591 - acc: 0.8134 -- iter: 768/805
Training Step: 130  | total loss: 4.44197 | time: 312.164s
| Adam | epoch: 010 | loss: 4.44197 - acc: 0.8071 | val_loss: 4.09348 - val_acc: 0.8222 -- iter: 805/805
--


In [8]:
model.save('3dconv-1-1-1-1-2.tflearn')

INFO:tensorflow:C:\Users\gabri\Documents\Python Scripts\Project\3dconv-1-1-1-1-1.tflearn is not in all_model_checkpoint_paths. Manually adding it.
